<a href="https://colab.research.google.com/github/khyatinayak/breast-cancer-detection/blob/main/Breast_Cancer_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import all the dependencies**

In [ ]:
import torch
from torch import nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

**Device** **Configuration**

In [ ]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

Using Device: cuda


#Data Collection and Preprocessing

In [ ]:
#load the breast cancer dataset
data=load_breast_cancer()
X=data.data
y=data.target
X[:5],y[:5]

(array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
         3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
         8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
         3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
         1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
         8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
         3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
         1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
         1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
         1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
         4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
         2.250e-02, 4.5

In [ ]:
#Split the dataset into training data and testing data
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X.shape, X_train.shape, X_test.shape

((569, 30), (455, 30), (114, 30))

In [ ]:
#standardize the data using Standard Scaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
#convert data to pytorch tensor and move it to GPU
X_train= torch.tensor(X_train, dtype=torch.float32).to(device)
y_train= torch.tensor(y_train, dtype=torch.float32).to(device)
X_test= torch.tensor(X_test, dtype=torch.float32).to(device)
y_test= torch.tensor(y_test, dtype=torch.float32).to(device)

#Neural Network Architecture

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self, input_shape:int, hidden_units:int, output_shape:int):
    super(NeuralNet, self).__init__()
    self.fc1=nn.Linear(input_shape, hidden_units)
    self.relu=nn.ReLU()
    self.fc2=nn.Linear(hidden_units, output_shape)
    self.sigmoid=nn.Sigmoid()

  def forward(self,x):
    out=self.fc1(x)
    out=self.relu(out)
    out=self.fc2(out)
    out=self.sigmoid(out)

    return out

In [ ]:
#define hyperparameters
input_shape=X_train.shape[1]
hidden_units=64
output_shape=1
num_of_epochs=1000

In [ ]:
#initialize the neural network and move it to GPU
model=NeuralNet(input_shape,hidden_units,output_shape).to(device)

In [ ]:
#define the loss function and optimizer
loss_fn=nn.BCELoss()
optimizer=optim.Adam(params=model.parameters(),lr=0.1)

def acc_score(y_true,y_pred):
  correct=torch.eq(y_true,y_pred).sum().item() #torch.eq checks how much equal are 2 parameters
  accuracy=(correct/len(y_pred))*100
  return accuracy

#Training the model

In [ ]:
for epoch in range(num_of_epochs):

    # Training mode
    model.train()

    # Forward pass
    y_preds = model(X_train).view(-1)  # Ensure y_preds has the same shape as y_train

    # Loss
    loss = loss_fn(y_preds, y_train)

    # Zero grad
    optimizer.zero_grad()

    # Loss backward
    loss.backward()

    # Step up optimizer
    optimizer.step()

    model.eval()
    with torch.inference_mode():
        # Forward pass
        test_pred = model(X_test).view(-1)  # Ensure test_pred has the same shape as y_test

        # Calculate loss
        test_loss = loss_fn(test_pred, y_test)

        # Calculate accuracy
        test_acc = acc_score(y_true=y_test, y_pred=test_pred > 0.5)

    # Print out what's happening every 10 epochs
    if epoch % 40 == 0:
        training_acc = acc_score(y_true=y_train, y_pred=y_preds > 0.5)  # Calculate training accuracy
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {training_acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.63759, Accuracy: 63.96% | Test loss: 0.12409, Test acc: 96.49%
Epoch: 40 | Loss: 0.00620, Accuracy: 99.78% | Test loss: 0.20124, Test acc: 97.37%
Epoch: 80 | Loss: 0.00049, Accuracy: 100.00% | Test loss: 0.24841, Test acc: 97.37%
Epoch: 120 | Loss: 0.00020, Accuracy: 100.00% | Test loss: 0.26644, Test acc: 97.37%
Epoch: 160 | Loss: 0.00012, Accuracy: 100.00% | Test loss: 1.00856, Test acc: 97.37%
Epoch: 200 | Loss: 0.00008, Accuracy: 100.00% | Test loss: 1.01240, Test acc: 97.37%
Epoch: 240 | Loss: 0.00006, Accuracy: 100.00% | Test loss: 1.01581, Test acc: 97.37%
Epoch: 280 | Loss: 0.00005, Accuracy: 100.00% | Test loss: 1.01900, Test acc: 97.37%
Epoch: 320 | Loss: 0.00004, Accuracy: 100.00% | Test loss: 1.02167, Test acc: 97.37%
Epoch: 360 | Loss: 0.00003, Accuracy: 100.00% | Test loss: 1.02413, Test acc: 97.37%
Epoch: 400 | Loss: 0.00003, Accuracy: 100.00% | Test loss: 1.02644, Test acc: 97.37%
Epoch: 440 | Loss: 0.00002, Accuracy: 100.00% | Test loss: 1.02854, Tes

In [ ]:
#Model in evaluation
model.eval()
with torch.inference_mode():
    # Forward pass
    test_pred = model(X_train).view(-1)  # Ensure test_pred has the same shape as y_test

    # Calculate accuracy
    test_acc = acc_score(y_true=y_train, y_pred=test_pred > 0.5)

    #accuracy
    print(f"Accuracy of training data: {test_acc:.2f}")

Accuracy of training data: 100.00


In [ ]:
#Model in evaluation
model.eval()
with torch.inference_mode():
    # Forward pass
    test_pred = model(X_test).view(-1)  # Ensure test_pred has the same shape as y_test

    # Calculate accuracy
    test_acc = acc_score(y_true=y_test, y_pred=test_pred > 0.5)

    #accuracy
    print(f"Accuracy of test data: {test_acc:.2f}")

Accuracy of test data: 97.37
